In [9]:
import csv

f = open('accommodation.csv', 'r', encoding='utf-8-sig')
rdr = csv.DictReader(f)
data = []
for row in rdr:
    if len(row['Address']) > 0:
        data.append(row)
f.close()

In [12]:
import requests
import json

for row in data:
    res = requests.get('https://dapi.kakao.com/v2/local/search/address.json', \
                       params={'query': row['Address']}, headers={'Authorization': 'KakaoAK ddaaef4912b0e162f6c1e2edf20b8ddf'})
    info = json.loads(res.text)
    if len(info['documents']) > 0:
        row['adm_code'] = info['documents'][0]['address']['h_code']

In [ ]:
f = open('accommodation_code.csv', 'w', encoding='utf-8', newline='')
wr = csv.DictWriter(f, data[0].keys())
wr.writeheader()
for row in data:
    wr.writerow(row)
f.close()

In [2]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [7]:
df = sqlContext.read.csv('accommodation_code.csv', encoding='utf-8', header=True)
df.show()

+----------+--------------------+
|  adm_code|             Address|
+----------+--------------------+
|1117068500|서울특별시 용산구 대사관로34길...|
|1117068500|서울특별시 용산구 한남대로21길...|
|1117052000|서울특별시 용산구 신흥로 41-...|
|1117057000|서울특별시 용산구 원효로41길 ...|
|1117055500|서울특별시 용산구 청파로57길 ...|
|1117056000|서울특별시 용산구 원효로53길 ...|
|1117062500|서울특별시 용산구 한강대로40길...|
|1117062500|서울특별시 용산구 한강대로15길...|
|1117062500|서울특별시 용산구 한강대로15길...|
|1117053000|서울특별시 용산구 후암로57길 ...|
|1117053000|서울특별시 용산구 후암로57길 ...|
|1117062500|서울특별시 용산구 한강대로62나...|
|1117064000|서울특별시 용산구 이촌로 16-...|
|1117053000|서울특별시 용산구 한강대로84길...|
|1117051000|서울특별시 용산구 두텁바위로 8...|
|1117055500|서울특별시 용산구 효창원로 22...|
|1117055500|서울특별시 용산구 만리재로36길...|
|1117062500|서울특별시 용산구 한강대로21길...|
|1117062500|서울특별시 용산구 한강대로17길...|
|1117062500|서울특별시 용산구 한강대로21길...|
+----------+--------------------+
only showing top 20 rows



In [31]:
df_null = df.filter(df['adm_code'].isNull())
df_null.show(150, False)

+--------+---------------------------------------------+
|adm_code|Address                                      |
+--------+---------------------------------------------+
|null    |서울특별시 용산구 한강대로 305 1~20층 (갈월동)               |
|null    |서울특별시 중구 명동8길 36 4~6층 (명동2가)                 |
|null    |서울특별시 중구 장충단로 213 3~7층 (광희동2가)               |
|null    |서울특별시 중구 남대문로 9 4~20층 (남대문로4가)               |
|null    |서울특별시 중구 퇴계로 52 4~18층 (회현동1가)                |
|null    |서울특별시 중구 동호로 359 2~20층 (을지로5가)               |
|null    |서울특별시 동대문구 고산자로62길 3 2~3층 (제기동)              |
|null    |서울특별시 동대문구 회기로31길 1 1~3층 (휘경동)               |
|null    |서울특별시 종로구 율곡로10길 17 (권농동)                    |
|null    |서울특별시 종로구 수표로26길 10 (돈의동)                    |
|null    |서울특별시 종로구 새문안로 97 10~25층 (당주동)               |
|null    |서울특별시 종로구 종로8길 15 4~5층 (관철동)                 |
|null    |서울특별시 중구 세종대로2길 11 (남대문로5가)                  |
|null    |서울특별시 종로구 돈화문로11나길 63 4~6층 (익선동)             |
|null    |서울특별시 중구 남대문시장길 61 4~

In [39]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
import requests
import json

def fixCode(addr):
    splited = addr.split(' ')
    if len(splited) >= 4:
        res = requests.get('https://dapi.kakao.com/v2/local/search/address.json', \
                           params={'query': '%s %s %s %s' % (splited[0], splited[1], splited[2], splited[3])},
                           headers={'Authorization': 'KakaoAK ddaaef4912b0e162f6c1e2edf20b8ddf'})
        info = json.loads(res.text)
        if len(info['documents']) > 0:
            return info['documents'][0]['address']['h_code']
    else:
        return None

fix_udf = udf(fixCode, StringType())
df_fixed = df_null.withColumn('adm_code', fix_udf(df_null['Address']))
df_fixed.show()

+----------+--------------------+
|  adm_code|             Address|
+----------+--------------------+
|1117053000|서울특별시 용산구 한강대로 30...|
|1114055000|서울특별시 중구 명동8길 36 ...|
|1114059000|서울특별시 중구 장충단로 213...|
|1114052000|서울특별시 중구 남대문로 9 4...|
|1114054000|서울특별시 중구 퇴계로 52 4...|
|1114060500|서울특별시 중구 동호로 359 ...|
|1123054500|서울특별시 동대문구 고산자로62...|
|1123072000|서울특별시 동대문구 회기로31길...|
|      null|서울특별시 종로구 율곡로10길 ...|
|      null|서울특별시 종로구 수표로26길 ...|
|1111053000|서울특별시 종로구 새문안로 97...|
|1111061500|서울특별시 종로구 종로8길 15...|
|      null|서울특별시 중구 세종대로2길 1...|
|1111061500|서울특별시 종로구 돈화문로11나...|
|1114054000|서울특별시 중구 남대문시장길 6...|
|1114055000|서울특별시 중구 명동8가길 51...|
|1114059000|서울특별시 중구 퇴계로 245 ...|
|1114059000|서울특별시 중구 을지로 255 ...|
|1114057000|서울특별시 중구 필동로8길 77...|
|      null|서울특별시 도봉구 도봉로 417...|
+----------+--------------------+
only showing top 20 rows



In [40]:
df = df.filter(df['adm_code'].isNotNull()).union(df_fixed)
df.show()

+----------+--------------------+
|  adm_code|             Address|
+----------+--------------------+
|1117068500|서울특별시 용산구 대사관로34길...|
|1117068500|서울특별시 용산구 한남대로21길...|
|1117052000|서울특별시 용산구 신흥로 41-...|
|1117057000|서울특별시 용산구 원효로41길 ...|
|1117055500|서울특별시 용산구 청파로57길 ...|
|1117056000|서울특별시 용산구 원효로53길 ...|
|1117062500|서울특별시 용산구 한강대로40길...|
|1117062500|서울특별시 용산구 한강대로15길...|
|1117062500|서울특별시 용산구 한강대로15길...|
|1117053000|서울특별시 용산구 후암로57길 ...|
|1117053000|서울특별시 용산구 후암로57길 ...|
|1117062500|서울특별시 용산구 한강대로62나...|
|1117064000|서울특별시 용산구 이촌로 16-...|
|1117053000|서울특별시 용산구 한강대로84길...|
|1117051000|서울특별시 용산구 두텁바위로 8...|
|1117055500|서울특별시 용산구 효창원로 22...|
|1117055500|서울특별시 용산구 만리재로36길...|
|1117062500|서울특별시 용산구 한강대로21길...|
|1117062500|서울특별시 용산구 한강대로17길...|
|1117062500|서울특별시 용산구 한강대로21길...|
+----------+--------------------+
only showing top 20 rows



In [48]:
df_null = df.filter(df['adm_code'].isNull())
df_null.show(100, False)

+--------+----------------------------------+
|adm_code|Address                           |
+--------+----------------------------------+
|null    |서울특별시 종로구 율곡로10길 17 (권농동)         |
|null    |서울특별시 종로구 수표로26길 10 (돈의동)         |
|null    |서울특별시 중구 세종대로2길 11 (남대문로5가)       |
|null    |서울특별시 도봉구 도봉로 417 (쌍문동)           |
|null    |서울특별시 도봉구 도봉로 613 (쌍문동)           |
|null    |서울특별시 강북구 솔매로30길 3 (미아동)          |
|null    |서울특별시 중랑구 망우로 394 (망우동)           |
|null    |서울특별시 성북구 인촌로 7 (보문동1가)           |
|null    |서울특별시 동대문구 사가정로 23 (답십리동)         |
|null    |서울특별시 동대문구 왕산로36길 29-12 234층 (전농동)|
|null    |서울특별시 동대문구 답십리로1길 39-5 (전농동 뉴부림호텔)|
|null    |서울특별시 중랑구 면목로42길 13 (면목동)         |
|null    |서울특별시 중랑구 면목로42길 11-4 (면목동)       |
|null    |서울특별시 중랑구 면목로95길 9 (상봉동)          |
|null    |서울특별시 광진구 광나루로14길 21 (화양동)        |
|null    |서울특별시 광진구 천호대로 538 (군자동22)        |
|null    |서울특별시 성북구 인촌로30길 25 (안암동5가)       |
|null    |서울특별시 성북구 인촌로2길 47 (보문동4가)        |
|null    |서울특별시 성북구 인촌로7길 88 (동선동2

In [55]:
def fixCode2(addr):
    splited = addr.split(' ')
    if len(splited) >= 4:
        jibonData = requests.get('http://www.juso.go.kr/addrlink/addrLinkApi.do', \
                                params={'confmKey': 'U01TX0FVVEgyMDE5MDYwNzE1MzIwMDEwODc5MDI=', \
                                        'keyword': '%s %s %s %s' % (splited[0], splited[1], splited[2], splited[3]), \
                                        'resultType': 'json'})
        jibonDict = json.loads(jibonData.text)
        if len(jibonDict['results']['juso']) == 0:
            return None
        res = requests.get('https://dapi.kakao.com/v2/local/search/address.json', \
                           params={'query': jibonDict['results']['juso'][0]['jibunAddr']},
                           headers={'Authorization': 'KakaoAK ddaaef4912b0e162f6c1e2edf20b8ddf'})
        info = json.loads(res.text)
        if len(info['documents']) > 0:
            return info['documents'][0]['address']['h_code']
    else:
        return None

fix_udf = udf(fixCode2, StringType())
df_fixed = df_null.withColumn('adm_code', fix_udf(df_null['Address']))
df_fixed.persist()
df_fixed.show(100, False)

+----------+----------------------------------+
|adm_code  |Address                           |
+----------+----------------------------------+
|1111061500|서울특별시 종로구 율곡로10길 17 (권농동)         |
|null      |서울특별시 종로구 수표로26길 10 (돈의동)         |
|null      |서울특별시 중구 세종대로2길 11 (남대문로5가)       |
|null      |서울특별시 도봉구 도봉로 417 (쌍문동)           |
|null      |서울특별시 도봉구 도봉로 613 (쌍문동)           |
|1130555500|서울특별시 강북구 솔매로30길 3 (미아동)          |
|null      |서울특별시 중랑구 망우로 394 (망우동)           |
|null      |서울특별시 성북구 인촌로 7 (보문동1가)           |
|1123060000|서울특별시 동대문구 사가정로 23 (답십리동)         |
|null      |서울특별시 동대문구 왕산로36길 29-12 234층 (전농동)|
|null      |서울특별시 동대문구 답십리로1길 39-5 (전농동 뉴부림호텔)|
|null      |서울특별시 중랑구 면목로42길 13 (면목동)         |
|null      |서울특별시 중랑구 면목로42길 11-4 (면목동)       |
|1126059000|서울특별시 중랑구 면목로95길 9 (상봉동)          |
|1121571000|서울특별시 광진구 광나루로14길 21 (화양동)        |
|null      |서울특별시 광진구 천호대로 538 (군자동22)        |
|null      |서울특별시 성북구 인촌로30길 25 (안암동5가)       |
|null      |서울특별시 성북구 인촌로2길 47 (보문동4가)  

In [56]:
df = df.filter(df['adm_code'].isNotNull()).union(df_fixed).select('Address', 'adm_code')
df.show()

+--------------------+----------+
|             Address|  adm_code|
+--------------------+----------+
|서울특별시 용산구 대사관로34길...|1117068500|
|서울특별시 용산구 한남대로21길...|1117068500|
|서울특별시 용산구 신흥로 41-...|1117052000|
|서울특별시 용산구 원효로41길 ...|1117057000|
|서울특별시 용산구 청파로57길 ...|1117055500|
|서울특별시 용산구 원효로53길 ...|1117056000|
|서울특별시 용산구 한강대로40길...|1117062500|
|서울특별시 용산구 한강대로15길...|1117062500|
|서울특별시 용산구 한강대로15길...|1117062500|
|서울특별시 용산구 후암로57길 ...|1117053000|
|서울특별시 용산구 후암로57길 ...|1117053000|
|서울특별시 용산구 한강대로62나...|1117062500|
|서울특별시 용산구 이촌로 16-...|1117064000|
|서울특별시 용산구 한강대로84길...|1117053000|
|서울특별시 용산구 두텁바위로 8...|1117051000|
|서울특별시 용산구 효창원로 22...|1117055500|
|서울특별시 용산구 만리재로36길...|1117055500|
|서울특별시 용산구 한강대로21길...|1117062500|
|서울특별시 용산구 한강대로17길...|1117062500|
|서울특별시 용산구 한강대로21길...|1117062500|
+--------------------+----------+
only showing top 20 rows



In [58]:
import pandas as pd
df.toPandas().to_csv('accommodates_code.csv', encoding="utf-8")